In [2]:
import os
import numpy as np
import tensorflow as tf
import hls4ml
import matplotlib.pyplot as plt



2024-06-22 21:29:01.031580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 21:29:01.110829: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


/home/dmitsas/anaconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


In [3]:
it_min = -128
it_max = 128
precision = 8

In [4]:
%%time
for pr in range(16,17,1):
    for i in range(it_min,it_max,1):
        if not(os.path.exists('./LUT_DSP/prec_' + str(pr) + "/it_" + str(i) + '/')):
            os.makedirs('./LUT_DSP/prec_' + str(pr) + "/it_" + str(i))
        with open('./LUT_DSP/prec_' + str(pr) + "/it_" + str(i) + "/mult.cpp",'w' ) as f:
            f.write("""#include <iostream> 
    #include <ap_int.h> 
    extern "C" { 
    void mult(ap_int<""" + str(pr) + """> bitnum,ap_int<""" + str(pr + 8) + """> *out){ 
    ap_int<""" + str(pr + 8) + """> tmp; 
    tmp =""" + str(i)+""" * bitnum;
    *out = tmp;
    }
    }"""
    )

CPU times: user 10.3 ms, sys: 4.07 ms, total: 14.4 ms
Wall time: 14 ms


In [5]:
%%time
for pr in range (16,17,1):
    for i in range(it_min,it_max,1):
        with open('./LUT_DSP/prec_' + str(pr) + '/run_test.tcl','w') as w:
            w.write("""set XPART xc7z007s-clg225-2
    set CSIM 0
    set CSYNTH 1
    set COSIM 0
    set VIVADO_SYN 0
    set VIVADO_IMPL 0 
    set PROJ mult.prj
    \nset SOLN solution1
    set CLKP 3.33
    open_project -reset $PROJ""" +
    "\nadd_files ./mult.cpp"
    """\nset_top mult
    open_solution -reset $SOLN
    set_part $XPART
    create_clock -period $CLKP
    config_op mul -impl dsp
    set_clock_uncertainty 0.10


    if {$CSYNTH == 1} {
      csynth_design
    }

    exit"""
    )
        w.close()
    

        os.system('bash ./LUT_DSP/run_test.sh ' + str(pr) + " " + str(i))
    for rem in range(2,pr + 1):
        for k in range(it_min,it_max,1):
            os.system('rm -r ' + './LUT_DSP/prec_' + str(rem)  + "/it_" + str(k) + '/mult.prj/solution1/.autopilot')


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2022.1 (64-bit)
  **** SW Build 3526262 on Mon Apr 18 15:47:01 MDT 2022
  **** IP Build 3524634 on Mon Apr 18 20:55:01 MDT 2022
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vitis_HLS/2022.1/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vitis_HLS/2022.1/bin/unwrapped/lnx64.o/vitis_hls'
INFO: [HLS 200-10] For user 'dmitsas' on host 'skylla' (Linux_x86_64 version 5.4.0-182-generic) on Sat Jun 22 21:29:07 EEST 2024
INFO: [HLS 200-10] On os Ubuntu 20.04 LTS
INFO: [HLS 200-10] In directory '/home/dmitsas/Downloads/notebooks/lut_mul/LUT_DSP/prec_16/it_-128'
Sourcing Tcl script '../run_test.tcl'
INFO: [HLS 200-1510] Running: open_project -reset mult.prj 
INFO: [HLS 200-10] Creating and opening project '/home/dmitsas/Downloads/notebooks/lut_mul/LUT_DSP/prec_16/it_-128/mult.prj'.
INFO: [HLS 200-1510] Running: add_files ./mult.cpp 
INFO: [HLS 200-10] Adding design 

INFO: [HLS 200-1611] Setting target device to 'xc7z007s-clg225-2'
INFO: [HLS 200-1510] Running: create_clock -period 3.33 
INFO: [SYN 201-201] Setting up clock 'default' with a period of 3.33ns.
INFO: [HLS 200-1510] Running: config_op mul -impl dsp 
INFO: [HLS 200-1445] Configure operator 'mul' with implementation style 'dsp'.
INFO: [HLS 200-1510] Running: set_clock_uncertainty 0.10 
INFO: [SYN 201-201] Setting up clock 'default' with an uncertainty of 0.1ns.
INFO: [HLS 200-1510] Running: csynth_design 
INFO: [HLS 200-111] Finished File checks and directory preparation: CPU user time: 0 seconds. CPU system time: 0 seconds. Elapsed time: 0 seconds; current allocated memory: 1.458 GB.
INFO: [HLS 200-10] Analyzing design file './mult.cpp' ... 
INFO: [HLS 200-111] Finished Source Code Analysis and Preprocessing: CPU user time: 4.53 seconds. CPU system time: 0.72 seconds. Elapsed time: 4.5 seconds; current allocated memory: 460.602 MB.
INFO: [HLS 200-777] Using interface defaults for 'Vivad

INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'mult' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-500] Setting interface mode on port 'mult/bitnum' to 'ap_none'.
INFO: [RTGEN 206-500] Setting interface mode on port 'mult/out_r' to 'ap_vld'.
INFO: [RTGEN 206-500] Setting interface mode on function 'mult' to 'ap_ctrl_hs'.
INFO: [RTGEN 206-100] Finished creating RTL model for 'mult'.
INFO: [HLS 200-111] Finished Creating RTL model: CPU user time: 0 seconds. CPU system time: 0.01 seconds. Elapsed time: 0.01 seconds; current allocated memory: 497.672 MB.
INFO: [HLS 200-111] Finished Generating all RTL models: CPU user time: 0.12 seconds. CPU system time: 0 seconds. Elapsed time: 0.12 seconds; current allocated memory: 497.672 MB.
INFO: [HLS 200-111] Finished Updating report files: CPU user time: 0.1 seconds. CPU system time: 0.01 seconds. Elapsed tim


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2022.1 (64-bit)
  **** SW Build 3526262 on Mon Apr 18 15:47:01 MDT 2022
  **** IP Build 3524634 on Mon Apr 18 20:55:01 MDT 2022
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vitis_HLS/2022.1/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vitis_HLS/2022.1/bin/unwrapped/lnx64.o/vitis_hls'
INFO: [HLS 200-10] For user 'dmitsas' on host 'skylla' (Linux_x86_64 version 5.4.0-182-generic) on Sat Jun 22 21:29:57 EEST 2024
INFO: [HLS 200-10] On os Ubuntu 20.04 LTS
INFO: [HLS 200-10] In directory '/home/dmitsas/Downloads/notebooks/lut_mul/LUT_DSP/prec_16/it_-124'
Sourcing Tcl script '../run_test.tcl'
INFO: [HLS 200-1510] Running: open_project -reset mult.prj 
INFO: [HLS 200-10] Creating and opening project '/home/dmitsas/Downloads/notebooks/lut_mul/LUT_DSP/prec_16/it_-124/mult.prj'.
INFO: [HLS 200-1510] Running: add_files ./mult.cpp 
INFO: [HLS 200-10] Adding design 

INFO: [HLS 200-1611] Setting target device to 'xc7z007s-clg225-2'
INFO: [HLS 200-1510] Running: create_clock -period 3.33 
INFO: [SYN 201-201] Setting up clock 'default' with a period of 3.33ns.
INFO: [HLS 200-1510] Running: config_op mul -impl dsp 
INFO: [HLS 200-1445] Configure operator 'mul' with implementation style 'dsp'.
INFO: [HLS 200-1510] Running: set_clock_uncertainty 0.10 
INFO: [SYN 201-201] Setting up clock 'default' with an uncertainty of 0.1ns.
INFO: [HLS 200-1510] Running: csynth_design 
INFO: [HLS 200-111] Finished File checks and directory preparation: CPU user time: 0.01 seconds. CPU system time: 0 seconds. Elapsed time: 0 seconds; current allocated memory: 1.458 GB.
INFO: [HLS 200-10] Analyzing design file './mult.cpp' ... 
INFO: [HLS 200-111] Finished Source Code Analysis and Preprocessing: CPU user time: 4.61 seconds. CPU system time: 0.77 seconds. Elapsed time: 4.5 seconds; current allocated memory: 460.602 MB.
INFO: [HLS 200-777] Using interface defaults for 'Vi

...

rm: cannot remove './LUT_DSP/prec_14/it_3/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_4/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_5/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_6/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_7/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_8/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_9/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_10/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_11/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_12/mult.prj/solution1/.autopilot': No such file or directo

rm: cannot remove './LUT_DSP/prec_14/it_122/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_123/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_124/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_125/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_126/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_14/it_127/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-128/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-127/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-126/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-125/mult.prj/solution1/.autopilot': No

rm: cannot remove './LUT_DSP/prec_15/it_-15/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-14/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-13/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-12/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-11/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-10/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-9/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-8/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-7/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_-6/mult.prj/solution1/.autopilot': No such fi

rm: cannot remove './LUT_DSP/prec_15/it_104/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_105/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_106/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_107/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_108/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_109/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_110/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_111/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_112/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_DSP/prec_15/it_113/mult.prj/solution1/.autopilot': No suc

CPU times: user 6.3 s, sys: 2.3 s, total: 8.6 s
Wall time: 54min 8s


In [9]:
%%time
for pr in range(2,8,1):
    for i in range(it_min,it_max,1):
        os.system('rm -r ' + './LUT_DSP/prec_' + str(pr)  + "/it_" + str(i) + '/mult.prj/solution1/.autopilot')

rm: cannot remove './LUT_sim/prec_2/it_-128/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-127/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-126/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-125/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-124/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-123/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-122/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-121/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-120/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-119/mult.prj/solution1/.autopilot': No suc

rm: cannot remove './LUT_sim/prec_2/it_-6/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-5/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-4/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-3/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-2/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_-1/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_0/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_1/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_2/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_3/mult.prj/solution1/.autopilot': No such file or directory
rm: 

rm: cannot remove './LUT_sim/prec_2/it_122/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_123/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_124/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_125/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_126/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_2/it_127/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_-128/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_-127/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_-126/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_-125/mult.prj/solution1/.autopilot': No such file

rm: cannot remove './LUT_sim/prec_3/it_-5/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_-4/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_-3/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_-2/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_-1/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_0/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_1/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_2/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_3/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_4/mult.prj/solution1/.autopilot': No such file or directory
rm: c

rm: cannot remove './LUT_sim/prec_3/it_123/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_124/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_125/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_126/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_3/it_127/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_4/it_-128/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_4/it_-127/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_4/it_-126/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_4/it_-125/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_4/it_-124/mult.prj/solution1/.autopilot': No such fil

...

rm: cannot remove './LUT_sim/prec_6/it_-16/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-15/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-14/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-13/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-12/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-11/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-10/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-9/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-8/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_-7/mult.prj/solution1/.autopilot': No such file or dir

rm: cannot remove './LUT_sim/prec_6/it_110/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_111/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_112/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_113/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_114/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_115/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_116/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_117/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_118/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_6/it_119/mult.prj/solution1/.autopilot': No such file or 

rm: cannot remove './LUT_sim/prec_7/it_-24/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-23/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-22/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-21/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-20/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-19/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-18/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-17/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-16/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_-15/mult.prj/solution1/.autopilot': No such file or 

CPU times: user 348 ms, sys: 416 ms, total: 764 ms
Wall time: 4.97 s


rm: cannot remove './LUT_sim/prec_7/it_102/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_103/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_104/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_105/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_106/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_107/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_108/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_109/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_110/mult.prj/solution1/.autopilot': No such file or directory
rm: cannot remove './LUT_sim/prec_7/it_111/mult.prj/solution1/.autopilot': No such file or 